<a href="https://colab.research.google.com/github/BlindCrowA/Human-Pose-Recognition/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe tensorflow

In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model

#import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt


mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

In [ ]:
def process_image(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Получаем координаты всех точек
        landmarks = results.pose_landmarks.landmark
        # Центрируем относительно базы позвоночника (точка с индексом 0)
        center_x, center_y, center_z = landmarks[0].x, landmarks[0].y, landmarks[0].z
        relative_landmarks = []
        for lm in landmarks:
            relative_landmarks.extend([
                lm.x - center_x,
                lm.y - center_y,
                lm.z - center_z,
            ])
        return relative_landmarks
    else:
        return None


def create_dataset(directory):
    data = []
    labels = []
    for label in os.listdir(directory):
        class_path = os.path.join(directory, label)
        if not os.path.isdir(class_path):
            continue
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            landmarks = process_image(img_path)
            if landmarks:
                data.append(landmarks)
                labels.append(label)
    return np.array(data), np.array(labels)



In [ ]:

# Укажите путь к директории с данными
data_dir = "/content/dataset"
X, y = create_dataset(data_dir)

# Сохранение в файл
df = pd.DataFrame(X)
df['label'] = y
df.to_csv("pose_dataset.csv", index=False)


In [ ]:
# Загрузка данных
dataset = pd.read_csv("pose_dataset.csv")
X = dataset.iloc[:, :-1].values  # Все столбцы, кроме последнего
y = dataset['label'].values     # Метки классов

# Преобразование меток в числовой формат
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Разделение данных на обучающие и тестовые
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание модели
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(set(y)), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# Callback для ранней остановки
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Метрика, которую отслеживаем
    patience=20,          # Количество эпох без улучшения
    restore_best_weights=True  # Восстановить лучшие веса
)


# Обучение модели
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)
model.save("pose_classification_model.keras")



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 116ms/step - accuracy: 0.1997 - loss: 1.6212 - val_accuracy: 0.0000e+00 - val_loss: 1.6370
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.2100 - loss: 1.5910 - val_accuracy: 0.0000e+00 - val_loss: 1.6091
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.2753 - loss: 1.5505 - val_accuracy: 0.0952 - val_loss: 1.5906
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.3388 - loss: 1.5172 - val_accuracy: 0.1429 - val_loss: 1.5731
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.3941 - loss: 1.4845 - val_accuracy: 0.1905 - val_loss: 1.5545
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3568 - loss: 1.4678 - val_accuracy: 0.1905 - val_loss: 1.5432
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4736 - loss: 1.4253 - val_accuracy: 0.1905 - val_loss: 1.5345
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5232 - loss: 1.3709 - val_accuracy: 0.1905 - 

In [ ]:

# Оценка модели
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Точность: {accuracy:.2f}")

# Построение графика потерь
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('loss_plot.png')  # Сохранение графика потерь
plt.close()

# Построение графика точности

plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('accuracy_plot.png')  # Сохранение графика точности
plt.close()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.9615 - loss: 0.2177
Точность: 0.96


In [ ]:
# Конвертация в TFLite формат
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Сохранение файла
with open("/content/pose_classification_model.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmp4vsif8o7'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 99), dtype=tf.float32, name='keras_tensor_35')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  136721567876544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136721477168896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136721477171184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136721339090016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136721339091424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136721339090720: TensorSpec(shape=(), dtype=tf.resource, name=None)
